# Stop detector

Yeah karena datanya sangat banyak stay nya mari kita hempaskan mereka

In [1]:
def create_plot(data):
    # Ambil baris pertama dari DataFrame
    first_row = data.iloc[0]

    # Ambil nilai latitude dan longitude dari baris pertama
    latitude = first_row['latitude']
    longitude = first_row['longitude']
    
    m = folium.Map(location=[latitude, longitude], zoom_start=25)
    
    # Add CircleMarkers for each point
    for index, row in data.iterrows():
        folium.CircleMarker(
            location=[row["latitude"], row["longitude"]],
            radius=5,  # Marker size
            color="blue",  # Marker color
            fill=True,
            fill_color="blue",  # Fill color of the marker
            fill_opacity=0.7,  # Opacity of the marker fill
            popup=f"User ID: {row['maid']}<br>Latitude: {row['latitude']}<br>Longitude: {row['longitude']}",
        ).add_to(m)
    
    return m

In [2]:
import folium
from shapely.geometry import Point, LineString

def create_plot_line(data):
    # Buat peta dengan lokasi awal berdasarkan rata-rata koordinat dari data linestring
    linestring_avg_lat = data["geometry"].apply(lambda x: x.centroid.y).mean()
    linestring_avg_lon = data["geometry"].apply(lambda x: x.centroid.x).mean()
    m = folium.Map(location=[linestring_avg_lat, linestring_avg_lon], zoom_start=15)

    # Tambahkan polyline ke peta menggunakan titik-titik dari linestring
    for linestring in data["geometry"]:
        folium.PolyLine(locations=list(linestring.coords), color='blue', weight=5, opacity=0.7).add_to(m)

    # Tambahkan CircleMarkers untuk setiap titik
    for index, row in data.iterrows():
        for point in row["geometry"].coords:
            folium.CircleMarker(
                location=[point[1], point[0]],  # Urutan koordinat diubah karena folium menggunakan (lat, lon)
                radius=5,  # Ukuran marker
                color="blue",  # Warna marker
                fill=True,
                fill_color="blue",  # Warna pengisian marker
                fill_opacity=0.7,  # Opasitas pengisian marker
                popup=f"User ID: {row['maid']}<br>Latitude: {point[1]}<br>Longitude: {point[0]}",
            ).add_to(m)

    return m

In [3]:
from shapely.geometry import LineString

def create_line(data):
    data = data.to_crs('EPSG:4326')
    data = data['geometry']
    # Buat peta dengan lokasi awal berdasarkan rata-rata koordinat dari data linestring
    avg_lat = data.apply(lambda x: x.centroid.y).mean()
    avg_lon = data.apply(lambda x: x.centroid.x).mean()
    m = folium.Map(location=[avg_lat, avg_lon], zoom_start=15)

    # Tambahkan polyline untuk setiap linestring
    for linestring in data:
        coordinates = [(lat, lon) for lon, lat in linestring.coords]
        folium.PolyLine(locations=coordinates, color='purple', weight=8, opacity=0.7).add_to(m)

    # Tambahkan GeoJson dari data GeoPandas
    folium.GeoJson(data.to_json(), name='Garis Jalan').add_to(m)

    return m

In [4]:
def bar_chart_traj(data):
    data = data.copy()
    df = pd.DataFrame(data)
    
    if 'tanggal' in df.columns:
        # Menghitung jumlah data berdasarkan tanggal
        jumlah_data_per_tanggal = df.groupby('tanggal').size()
    
    else:
        # Ambil tanggal dari kolom 'start_t' dan tambahkan sebagai kolom baru 'tanggal'
        df['tanggal'] = df['start_t'].dt.date
    
        # Menghitung jumlah data berdasarkan tanggal
        jumlah_data_per_tanggal = df.groupby('tanggal').size()
        
    # Plot bar chart
    plt.figure(figsize=(10, 6))
    jumlah_data_per_tanggal.plot(kind='bar', color='skyblue')
    plt.title(f'Jumlah Data per Tanggal: {jumlah_data_per_tanggal.sum()}')
    plt.xlabel('Tanggal')
    plt.ylabel('Jumlah Data Trajectory')
    plt.xticks(rotation=75)
    plt.grid(axis='y', linestyle='--', alpha=0.7)
    plt.tight_layout()
    plt.show()

In [5]:
def bar_chart_maid(data):
    data = data.copy()
    df = pd.DataFrame(data)
    
    if 'tanggal' in df.columns:
        # Menghitung jumlah maid berdasarkan tanggal
        jumlah_maid_unik_per_hari = df.groupby('tanggal')['maid'].nunique()
    
    else:
        # Ambil tanggal dari kolom 'start_t' dan tambahkan sebagai kolom baru 'tanggal'
        df['tanggal'] = df['start_t'].dt.date
        
        # Menghitung jumlah maid berdasarkan tanggal
        jumlah_maid_unik_per_hari = df.groupby('tanggal')['maid'].nunique()


    # Plotting
    plt.figure(figsize=(10, 6))
    jumlah_maid_unik_per_hari.plot(kind='bar', color='skyblue')
    plt.title(f'Total Jumlah Maid Unik per Hari: {jumlah_maid_unik_per_hari.sum()}')
    plt.xlabel('Tanggal')
    plt.ylabel('Jumlah Maid Unik')
    plt.xticks(rotation=75)
    plt.grid(axis='y', linestyle='--', alpha=0.7)
    plt.tight_layout()
    plt.show()

In [6]:
import pandas as pd
import geopandas as gpd
import movingpandas as mpd
import shapely as shp
import hvplot.pandas 
import matplotlib.pyplot as plt
import geoviews
import folium
import mapclassify

from geopandas import GeoDataFrame, read_file
from shapely.geometry import Point, LineString, Polygon
from datetime import datetime, timedelta
from holoviews import opts

import warnings
warnings.filterwarnings('ignore')

plot_defaults = {'linewidth':5, 'capstyle':'round', 'figsize':(9,3), 'legend':True}
opts.defaults(opts.Overlay(active_tools=['wheel_zoom'], frame_width=500, frame_height=400))

mpd.show_versions()


MovingPandas 0.17.2

SYSTEM INFO
-----------
python     : 3.10.13 (main, Sep 11 2023, 13:44:35) [GCC 11.2.0]
executable : /raid/arkham/miniconda3/envs/arkham/bin/python3
machine    : Linux-4.15.0-142-generic-x86_64-with-glibc2.27

GEOS, GDAL, PROJ INFO
---------------------
GEOS       : None
GEOS lib   : None
GDAL       : 3.6.2
GDAL data dir: /raid/arkham/miniconda3/envs/arkham/lib/python3.10/site-packages/fiona/gdal_data
PROJ       : 9.3.1
PROJ data dir: /raid/arkham/miniconda3/envs/arkham/lib/python3.10/site-packages/pyproj/proj_dir/share/proj

PYTHON DEPENDENCIES
-------------------
geopandas  : 0.14.3
pandas     : 2.1.4
fiona      : 1.9.5
numpy      : 1.23.5
shapely    : 2.0.1
rtree      : 1.2.0
pyproj     : 3.6.1
matplotlib : 3.5.3
mapclassify: 2.6.1
geopy      : 2.4.0
holoviews  : 1.18.3
hvplot     : 0.9.2
geoviews   : 1.11.1
stonesoup  : None


In [7]:
df = pd.read_csv('../../../DataGPS_Malioboro/filter2_maljan_nov.csv')
df_grouped = df.groupby(['maid', 'tanggal'])
tdf = mpd.TrajectoryCollection(df, traj_id_col='maid', t='datetime_wib', y='latitude', x='longitude')
tdf

TrajectoryCollection with 1838 trajectories

In [8]:
#record points
len(df)

40693

In [9]:
#total maid unik per bulan
len(df.groupby(['maid']))

1942

In [ ]:
#total maid unik per hari
bar_chart_maid(df)

In [88]:
index = 0
df_1min = pd.DataFrame()

while index < len(tdf):
    my_traj = tdf.trajectories[index]
    split = mpd.ObservationGapSplitter(my_traj).split(gap=timedelta(minutes=5))
    if len(split) > 1:
        split_df = split.to_traj_gdf()
        df_1min = df_1min._append(split_df, ignore_index=True)
    index += 1

In [89]:
df_1min['maid'] = df_1min['maid'].apply(lambda x: x.split('_')[0])

In [90]:
# total maid unik per sebulan
df_1min['maid'].nunique()

767

In [ ]:
# total maid unik per hari
bar_chart_maid(df_1min)

In [ ]:
# total trajectory
bar_chart_traj(df_1min)

In [91]:
# apply for traj length more than 0

df_1min_length = df_1min[df_1min['length'] > 0].reset_index(drop=True)
df_1min_length= df_1min_length.sort_values(by='length', ascending=False).reset_index(drop=True)
# df_1min_length = df_1min_length.to_crs('EPSG:3857')
# df_1min_length['length_meter'] = df_1min_length['geometry'].length

In [92]:
# total maid unik per sebulan
df_1min_length['maid'].nunique()

724

In [ ]:
# total maid unik per hari
bar_chart_maid(df_1min_length)

In [ ]:
# total trajectory
bar_chart_traj(df_1min_length)

In [93]:
# apply for traj length >5 
df_1min_length_5met = df_1min_length[df_1min_length['length']>5]

In [94]:
# total maid unik per sebulan
df_1min_length_5met['maid'].nunique()

420

In [ ]:
# total maid unik per hari
bar_chart_maid(df_1min_length_5met)

In [ ]:
# total trajectory
bar_chart_traj(df_1min_length_5met)

## Create points gdf

In [95]:
index = 0
points_1min = pd.DataFrame()

while index < len(tdf):
    my_traj = tdf.trajectories[index]
    split = mpd.ObservationGapSplitter(my_traj).split(gap=timedelta(minutes=5))
    if len(split) > 1:
        points_df = split.to_point_gdf()
        points_1min = points_1min._append(points_df)
    index += 1
points_1min.reset_index(drop=False, inplace=True)
points_1min['maid'] = points_1min['maid'].apply(lambda x: x.split('_')[0])

In [96]:
def create_points(points, traj):
    traj = traj.copy()
    points = points.copy()
    filtered_data = []
        
    for index, row in traj.iterrows():
        maid = row['maid']
        start_t = row['start_t']
        end_t = row['end_t']
        
        # Filter points from data based on maid and datetime range
        filtered = points[(points['maid'] == maid) & 
                          (points['datetime_wib'] >= start_t) & 
                          (points['datetime_wib'] <= end_t)]
        
        filtered_data.append(filtered)

    # Concatenate all filtered dataframes into one
    df_filtered = pd.concat(filtered_data, ignore_index=True)
    
    return df_filtered 

In [97]:
dfp_1min_length_5met = create_points(points_1min, df_1min_length_5met)

In [98]:
len(df[df['maid'] == 'b087a3ad-86ab-421b-bdd9-ac52382bd688'])
a = df[df['maid'] == 'b087a3ad-86ab-421b-bdd9-ac52382bd688']

In [101]:
df_1min_length[df_1min_length['maid'] == 'b087a3ad-86ab-421b-bdd9-ac52382bd688']

,maid,start_t,end_t,geometry,length,direction
427,b087a3ad-86ab-421b-bdd9-ac52382bd688,2021-11-01 14:23:32,2021-11-01 14:28:50,"LINESTRING (110.36550 -7.79426, 110.36550 -7.7...",54.963221,91.353331
511,b087a3ad-86ab-421b-bdd9-ac52382bd688,2021-11-01 18:04:08,2021-11-01 18:09:43,"LINESTRING (110.36597 -7.79425, 110.36588 -7.7...",40.564404,212.939462
541,b087a3ad-86ab-421b-bdd9-ac52382bd688,2021-11-01 17:48:08,2021-11-01 17:57:47,"LINESTRING (110.36591 -7.79419, 110.36592 -7.7...",36.715319,173.417005
933,b087a3ad-86ab-421b-bdd9-ac52382bd688,2021-11-01 15:49:37,2021-11-01 15:54:40,"LINESTRING (110.36548 -7.79403, 110.36547 -7.7...",14.557552,5.142031
1381,b087a3ad-86ab-421b-bdd9-ac52382bd688,2021-11-13 15:33:08,2021-11-13 15:38:21,"LINESTRING (110.36549 -7.79419, 110.36549 -7.7...",6.856871,0.000000
1432,b087a3ad-86ab-421b-bdd9-ac52382bd688,2021-11-01 16:55:30,2021-11-01 16:56:46,"LINESTRING (110.36595 -7.79429, 110.36596 -7.7...",6.351449,319.973810
1641,b087a3ad-86ab-421b-bdd9-ac52382bd688,2021-11-01 17:17:02,2021-11-01 17:19:41,"LINESTRING (110.36591 -7.79436, 110.36591 -7.7...",5.004369,0.000000
2241,b087a3ad-86ab-421b-bdd9-ac52382bd688,2021-11-01 17:38:11,2021-11-01 17:39:23,"LINESTRING (110.36593 -7.79439, 110.36593 -7.7...",2.803691,0.000000
3573,b087a3ad-86ab-421b-bdd9-ac52382bd688,2021-11-13 13:01:07,2021-11-13 13:01:57,"LINESTRING (110.36595 -7.79254, 110.36596 -7.7...",1.389697,37.085420
4454,b087a3ad-86ab-421b-bdd9-ac52382bd688,2021-11-13 13:56:08,2021-11-13 13:56:58,"LINESTRING (110.36562 -7.79476, 110.36562 -7.7...",0.831764,262.309146


In [100]:
df_1min_length_5met[df_1min_length_5met['maid'] == 'b087a3ad-86ab-421b-bdd9-ac52382bd688']

,maid,start_t,end_t,geometry,length,direction
427,b087a3ad-86ab-421b-bdd9-ac52382bd688,2021-11-01 14:23:32,2021-11-01 14:28:50,"LINESTRING (110.36550 -7.79426, 110.36550 -7.7...",54.963221,91.353331
511,b087a3ad-86ab-421b-bdd9-ac52382bd688,2021-11-01 18:04:08,2021-11-01 18:09:43,"LINESTRING (110.36597 -7.79425, 110.36588 -7.7...",40.564404,212.939462
541,b087a3ad-86ab-421b-bdd9-ac52382bd688,2021-11-01 17:48:08,2021-11-01 17:57:47,"LINESTRING (110.36591 -7.79419, 110.36592 -7.7...",36.715319,173.417005
933,b087a3ad-86ab-421b-bdd9-ac52382bd688,2021-11-01 15:49:37,2021-11-01 15:54:40,"LINESTRING (110.36548 -7.79403, 110.36547 -7.7...",14.557552,5.142031
1381,b087a3ad-86ab-421b-bdd9-ac52382bd688,2021-11-13 15:33:08,2021-11-13 15:38:21,"LINESTRING (110.36549 -7.79419, 110.36549 -7.7...",6.856871,0.000000
1432,b087a3ad-86ab-421b-bdd9-ac52382bd688,2021-11-01 16:55:30,2021-11-01 16:56:46,"LINESTRING (110.36595 -7.79429, 110.36596 -7.7...",6.351449,319.973810
1641,b087a3ad-86ab-421b-bdd9-ac52382bd688,2021-11-01 17:17:02,2021-11-01 17:19:41,"LINESTRING (110.36591 -7.79436, 110.36591 -7.7...",5.004369,0.000000


In [99]:
len(dfp_1min_length_5met[dfp_1min_length_5met['maid'] == 'b087a3ad-86ab-421b-bdd9-ac52382bd688'])
b = dfp_1min_length_5met[dfp_1min_length_5met['maid'] == 'b087a3ad-86ab-421b-bdd9-ac52382bd688']

## Stop detection

In [28]:
tc = mpd.TrajectoryCollection(dfp_1min_length_5met, 'maid', t='datetime_wib')

In [159]:
haha = mpd.TrajectoryCollection(dfp2_1min, 'maid', t='datetime_wib')

In [160]:
hihi = mpd.StopSplitter(haha).split(min_duration=timedelta(seconds=60), max_diameter=100)

In [161]:
hoho = hihi.to_point_gdf()
hoho.reset_index(drop=False, inplace=True)

In [162]:
hoho['maid'] = hoho['maid'].apply(lambda x: x.split('_')[0])
hehe = mpd.TrajectoryCollection(hoho, traj_id_col='maid', t='datetime_wib')

In [163]:
index = 0
xixi = pd.DataFrame()

while index < len(hehe):
    my_traj3 = hehe.trajectories[index]
    split3 = mpd.ObservationGapSplitter(my_traj3).split(gap=timedelta(minutes=5))
#     if len(split3) > 1:
    split_df3 = split3.to_traj_gdf()
    xixi = xixi._append(split_df3, ignore_index=True)
    index += 1

In [164]:
xixi

,maid,start_t,end_t,geometry,length,direction
0,0d21360c-6686-4e55-bdc9-7f64e5996255_0,2021-11-06 14:47:06,2021-11-06 14:47:56,"LINESTRING (110.36481 -7.80135, 110.36497 -7.8...",77.932381,123.483930
1,0d21360c-6686-4e55-bdc9-7f64e5996255_1,2021-11-06 15:04:37,2021-11-06 15:09:04,"LINESTRING (110.36646 -7.79011, 110.36603 -7.7...",251.609331,194.747486
2,0d21360c-6686-4e55-bdc9-7f64e5996255_3,2021-11-06 16:15:59,2021-11-06 16:16:59,"LINESTRING (110.36618 -7.79255, 110.36610 -7.7...",327.253507,353.501898
3,0d21360c-6686-4e55-bdc9-7f64e5996255_6,2021-11-06 22:20:00,2021-11-06 22:20:55,"LINESTRING (110.36566 -7.79346, 110.36585 -7.7...",463.508146,236.516567
4,0d21360c-6686-4e55-bdc9-7f64e5996255_7,2021-11-06 22:34:00,2021-11-06 22:37:54,"LINESTRING (110.36625 -7.79210, 110.36607 -7.7...",1055.539392,186.163194
...,...,...,...,...,...,...
154,f501c8a1-e251-4b8c-87fc-9a1a26d2e19d_78,2021-11-29 16:22:56,2021-11-29 16:26:25,"LINESTRING (110.36510 -7.79828, 110.36542 -7.7...",81.220384,25.864105
155,f501c8a1-e251-4b8c-87fc-9a1a26d2e19d_79,2021-11-30 08:40:38,2021-11-30 08:46:05,"LINESTRING (110.36574 -7.79351, 110.36573 -7.7...",599.619123,185.128686
156,f6edbc5b-10dd-4c24-b09d-beeb73ce105e_0,2021-11-08 15:40:03,2021-11-08 15:42:55,"LINESTRING (110.36592 -7.79213, 110.36592 -7.7...",465.708079,186.119033
157,f6edbc5b-10dd-4c24-b09d-beeb73ce105e_1,2021-11-18 12:01:12,2021-11-18 12:02:14,"LINESTRING (110.36600 -7.79136, 110.36600 -7.7...",242.191579,187.716026


In [29]:
split = mpd.StopSplitter(tc).split(min_duration=timedelta(seconds=60), max_diameter=100)

In [30]:
split

TrajectoryCollection with 316 trajectories

In [103]:
len(dfp_1min_length_5met)

21329

In [133]:
point_split = split.to_point_gdf()
point_split.reset_index(drop=False, inplace=True)

In [107]:
point_split['maid'] = point_split['maid'].apply(lambda x: x.split('_')[0])
point_split_tdf = mpd.TrajectoryCollection(point_split, traj_id_col='maid', t='datetime_wib')

In [108]:
index = 0
df2_1min = pd.DataFrame()

while index < len(point_split_tdf):
    my_traj2 = point_split_tdf.trajectories[index]
    split2 = mpd.ObservationGapSplitter(my_traj2).split(gap=timedelta(minutes=5))
    if len(split2) > 1:
        split_df2 = split2.to_traj_gdf()
        df2_1min = df2_1min._append(split_df2, ignore_index=True)
    index += 1

In [135]:
df2_1min['maid'] = df2_1min['maid'].apply(lambda x: x.split('_')[0])
person = df2_1min[df2_1min['maid'] == 'f6edbc5b-10dd-4c24-b09d-beeb73ce105e']

In [152]:
df2_1min.head(50)

,maid,start_t,end_t,geometry,length,direction
0,0d21360c-6686-4e55-bdc9-7f64e5996255,2021-11-06 14:47:06,2021-11-06 14:47:56,"LINESTRING (110.36481 -7.80135, 110.36497 -7.8...",77.932381,123.483930
1,0d21360c-6686-4e55-bdc9-7f64e5996255,2021-11-06 15:04:37,2021-11-06 15:09:04,"LINESTRING (110.36646 -7.79011, 110.36608 -7.7...",251.938133,194.747486
2,0d21360c-6686-4e55-bdc9-7f64e5996255,2021-11-06 15:17:06,2021-11-06 15:17:17,"LINESTRING (110.36618 -7.79263, 110.36575 -7.7...",59.604476,232.060414
3,0d21360c-6686-4e55-bdc9-7f64e5996255,2021-11-06 16:15:59,2021-11-06 16:16:59,"LINESTRING (110.36618 -7.79255, 110.36610 -7.7...",327.253507,353.501898
4,0d21360c-6686-4e55-bdc9-7f64e5996255,2021-11-06 17:06:08,2021-11-06 17:07:54,"LINESTRING (110.36623 -7.79232, 110.36626 -7.7...",31.041132,349.797415
5,0d21360c-6686-4e55-bdc9-7f64e5996255,2021-11-06 18:59:33,2021-11-06 19:03:00,"LINESTRING (110.36611 -7.79320, 110.36568 -7.7...",95.889843,215.032221
6,0d21360c-6686-4e55-bdc9-7f64e5996255,2021-11-06 22:20:00,2021-11-06 22:20:55,"LINESTRING (110.36566 -7.79346, 110.36585 -7.7...",463.508146,236.516567
7,0d21360c-6686-4e55-bdc9-7f64e5996255,2021-11-06 22:34:00,2021-11-06 22:37:54,"LINESTRING (110.36625 -7.79210, 110.36623 -7.7...",1056.321462,186.163194
8,0d21360c-6686-4e55-bdc9-7f64e5996255,2021-11-06 22:43:02,2021-11-06 22:43:55,"LINESTRING (110.36593 -7.79346, 110.36581 -7.7...",262.695552,62.113289
9,0d21360c-6686-4e55-bdc9-7f64e5996255,2021-11-07 00:32:27,2021-11-07 00:33:01,"LINESTRING (110.36620 -7.79129, 110.36626 -7.7...",76.537350,175.614855


In [121]:
person[0:1]

,maid,start_t,end_t,geometry,length,direction
0,0d21360c-6686-4e55-bdc9-7f64e5996255,2021-11-06 14:47:06,2021-11-06 14:47:56,"LINESTRING (110.36481 -7.80135, 110.36497 -7.8...",77.932381,123.48393


In [ ]:
# single trajectory
my_traj = split.trajectories[0]
traj_plot = my_traj.hvplot(title='Trajectory {}'.format(my_traj.id), line_width=7.0, tiles='CartoLight', color='slategray') 
traj_plot

In [ ]:
detector = mpd.TrajectoryStopDetector(my_traj)

In [ ]:
# Stop duration
stop_time_ranges = detector.get_stop_time_ranges(min_duration=timedelta(seconds=60), max_diameter=100)
for x in stop_time_ranges: 
    print(x)

In [ ]:
# stop points
stop_points = detector.get_stop_points(min_duration=timedelta(seconds=60), max_diameter=100)
stop_points

In [ ]:
# stop_point_plot = traj_plot * stop_points.hvplot(geo=True, size='duration_s', color='deeppink')
# stop_point_plot

In [ ]:
# Stop segments
stops = detector.get_stop_segments(min_duration=timedelta(seconds=60), max_diameter=100)
stops

In [ ]:
# stop_segment_plot = stop_point_plot * stops.hvplot(line_width=7.0, tiles=None, color='orange') 
# stop_segment_plot

## Stop detection for TrajectoryCollections

In [ ]:
detector = mpd.TrajectoryStopDetector(split)
stop_points = detector.get_stop_points(min_duration=timedelta(seconds=120), max_diameter=100)
len(stop_points)

In [ ]:
ax = tc.plot(figsize=(7,7))
stop_points.plot(ax=ax, color='red')

In [ ]:
tc = mpd.TrajectoryCollection(df_1min_length_5met, 'maid', 't')
tc

In [ ]:
type(df_1min_length_5met)

In [ ]:
df_1min_length_5met

## Map visualization

In [139]:
create_plot_line(person[2:3])

In [ ]:
create_line(df_1min_length_sorted[df_1min_length_sorted['start_t'] > '2021-12-22'])

In [ ]:
bar_chart_traj(df_1min_length_sorted)

In [ ]:
bar_chart_maid(df_1min_length_sorted)

In [ ]:
len(df_1min_length_sorted)